In [1]:
import sqlite3
from sodapy import Socrata
import math
# https://data.montgomerycountymd.gov/resource/s5gi-izpu.json

In [3]:
client = Socrata("data.montgomerycountymd.gov", None)
results = client.get("s5gi-izpu", limit=25000)

In [4]:
cats = []
for pet in results:
    if pet["animaltype"] == "CAT":
        cats.append(pet)

In [5]:
cat_database = '/home/maf/anaconda3/lib/python3.9/sqlite3/cat.db'
def create_connection(db_file):
    # create SQLite database, from https://www.sqlitetutorial.net/sqlite-python/creating-database/
    """ Create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Exception as e:
        print(e)
    finally:
        if conn:
            conn.close()

  # lets pretend sqilte can't handle JSON
  # create db
create_connection(cat_database)
  # connect
dbconn = sqlite3.connect(cat_database)
cur = dbconn.cursor()


In [6]:
cat_database = '/home/maf/anaconda3/lib/python3.9/sqlite3/cat.db'
FIELDS = ["id", "name", "color", "intake_date", "sex",
          "days_in_shelter", "size", "intake_subtype", "breed", "intake_condition",
          "date_of_birth", "intake_jurisdiction", "intake_type", "outcome_date", "outcome_subtype",
          "outcome_condition", "outcome_jurisdiction", "outcome_type"]
CAT_TABLE = """
CREATE TABLE IF NOT EXISTS cats_all 
(
    id TEXT,
    name TEXT,
    color TEXT,
    intake_date DATE,
    sex TEXT,
    days_in_shelter INT,
    size TEXT,
    intake_subtype TEXT,
    breed TEXT,
    intake_condition TEXT,
    date_of_birth DATE,
    intake_jurisdiction TEXT,
    intake_type TEXT,
    outcome_date DATE,
    outcome_subtype TEXT,
    outcome_condition TEXT,
    outcome_jurisdiction TEXT,
    outcome_type TEXT,
    updated_datetime DATETIME
);
"""

INSERT_STATEMENT = "INSERT INTO cats_all VALUES"

In [7]:
### create table
cur.execute(CAT_TABLE)
INSERT_COUNTS = 500
value_list = []

## loop over every 500
total_cats = len(cats)
times_around = int(math.ceil(total_cats / INSERT_COUNTS)) + 1

for loop_around in range(0, times_around):
    for cat in cats[loop_around*INSERT_COUNTS:(loop_around*INSERT_COUNTS)+INSERT_COUNTS]:
        this_list = []
        # parse the JSON
        for field in FIELDS:
            ## avoid TypeError: sequence item 1: expected string or Unicode, NoneType found
            if cat.get(field) is None:
                cat_field = "NULL"
            else:
                cat_field = cat.get(field).replace("'", "")
            this_list.append(cat_field)
        value_list.append("', '".join(this_list) + "', strftime('%Y-%m-%d %H-%M-%S','now')),")
    insert = "(\'" + "('".join(value_list)

## ensure it's empty
cur.execute("DELETE FROM cats_all")
## insert
cur.execute(INSERT_STATEMENT +  insert[:-1]  + ';')
# cur.close()

In [9]:
cur.execute("""SELECT * FROM cats_all""")
rows = cur.fetchall()
print("who is our first cat?")
print(rows[5])

who is our first cat?
('NULL', 'NULL', 'ORANGE TAB', 'NULL', 'N', 'NULL', 'NULL', 'NULL', 'DOMESTIC SH', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '2023-06-30 21-58-41')


In [10]:
cats = cur.execute(""" 
        SELECT size,
            CASE WHEN upper(color) = 'BLACK/BLK TABBY' THEN 'BLACK'
            WHEN upper(color) like 'BLK%' THEN 'BLACK OTHER'
            WHEN upper(color) like 'BLUE%' THEN 'BLUE'
            WHEN upper(color) like 'BRN %' THEN 'BROWN'
            WHEN upper(color) like 'BROWN' THEN 'BROWN'
            WHEN upper(color) like 'BUFF%' THEN 'ORANGE'
            WHEN upper(color) like 'ORG %' THEN 'ORANGE'
            WHEN upper(color) like 'ORANGE%' THEN 'ORANGE'
            WHEN upper(color) like 'WHITE' THEN 'WHITE'
            WHEN upper(color) like 'BLACK' THEN 'BLACK'
            WHEN upper(color) like 'TORTIE%' THEN 'TORTIE'
            WHEN upper(color) like 'WHITE/%' THEN 'WHITE MIXED'
            WHEN upper(color) like 'GRAY%' THEN 'GRAY'
            WHEN upper(color) like 'CALICO%' THEN 'CALICO'
            WHEN upper(color) like 'CHOC%' THEN 'CHOCOLATE'
            ELSE 'MIXED' end as color_group,
                count(*),
                avg(case when days_at_intake < 48 
                    then days_in_shelter - (48- days_at_intake)
                    else days_in_shelter 
                end) as days_in_shelter_normalized
            from (
            SELECT *, 
                julianday(intake_date) - julianday(date_of_birth) as days_at_intake,
                julianday(outcome_date) - julianday(intake_date) as days_in_shelter,
                julianday(outcome_date) - julianday(date_of_birth) as days_at_adoption
            FROM cats_all 
            where outcome_type = 'ADOPTION'
            and intake_condition = 'HEALTHY'
            )
            group by 1,2
                    """)


In [16]:
for cat in cats.fetchall():
    print(cat)
print(cat)    

{'animalid': 'A503628', 'intype': 'OWNER SUR', 'indate': '2023-04-15T00:00:00.000', 'petname': 'MARS', 'animaltype': 'CAT', 'petage': '1 YEAR 5 MONTHS', 'petsize': 'SMALL', 'color': 'BROWN', 'breed': 'DOMESTIC SH', 'sex': 'S', 'url': {'url': 'http://www.petharbor.com/get_image.asp?res=DETAIL&id=A503628&location=MONT'}}
